In [1]:
import json
import uuid
from functools import reduce, wraps

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML

from jinja2 import Template

# Smartsheet
import sys
from collections import defaultdict, namedtuple
import smartsheet

# Define a quartely time schema
years_in_model = 3
quarters = range(1, years_in_model * 4 + 1)
quarter_labels = ['Q' + str(q) for q in quarters]
#### STEP 1:  Calculate simple non-discounted cost-benefit cash flows for a single quarter with a delay ramp, and max
#### STEP 2:  Use those cash flows to find a discounted cash flow for each project using Net Present Value (NPV)
#### STEP 3:  Assemble the net present value of estimated future cash flows across projects in the portfolio
#### STEP 4:  Show a composite overall return, and let the user change paramaters, risk levels, timing, etc. to optimize and plan

# Cash Flow Analysis

### Unweighted Cash Flow

In [2]:
class CashFlow():
    def __init__(self, delay_qtrs, discount_rate, is_cost, max_amt, scale_up_qtrs,
                 function, name='', discounted=True, flow_id=uuid.uuid4(), tot_qtrs=12):
        if scale_up_qtrs < 2: 
            raise Exception('the total number of quarters must be at least one')

        self.delay_qtrs = delay_qtrs
        self.discounted = discounted
        self.discount_rate = discount_rate / 4 # annual discount rate -> quarterly
        self.function = function # Will interpret an instance according to this setting
        self.id = flow_id
        self.is_cost = is_cost
        self.max_amt = max_amt
        self.name = name
        self.scale_up_qtrs = scale_up_qtrs
        self.tot_qtrs = tot_qtrs  # TODO: rename to "period"
        
    def _sigmoid(self, x):
        """
        We define y at 95% max at end of delay and scale up period 
        y = .95L = L / (1 + e^-k(x_end - x_naught)) # https://en.wikipedia.org/wiki/Logistic_function
        For us, this means the scaling factor (k) is
        k = ln(1/.95 - 1)/((delay_qtrs + scale_up_qtrs/2) - (delay_qtrs + scale_up_qtrs))
        """
        # 1 / 2 scale_up_qtrs to place x_naught at vertical of sigmoid from after delay
        x_naught = self.delay_qtrs + self.scale_up_qtrs / 2
        x_end = self.delay_qtrs + self.scale_up_qtrs
        k = math.log(1/.95 - 1) / (x_naught - x_end)
        return self.max_amt / (1 + math.exp(-k * (x - x_naught)))
    
    def _linear(self, x):
        """y = mx + b. Units in amount (returned value) per quarter (x)"""
        m = self.max_amt / self.scale_up_qtrs
        b = -m * self.delay_qtrs
        return min(m * x + b, self.max_amt) # Never return more than max
    
    def _single(self, x):
        if x == self.delay_qtrs:
            return self.max_amt
        else:
            return 0
    
    def _step(self, x):
        return self.max_amt
 
    def _discounted_cash_flow(self, f):
        @wraps(f)
        def discounted_wrapper(quarter_n):
            return f(quarter_n) / (1 + self.discount_rate) ** quarter_n

        return discounted_wrapper
    
    def _calculate_qtr(self, f):
        values = []
        discounted_f = self._discounted_cash_flow(f) if self.discounted else f
        for quarter_n in range(0, self.tot_qtrs):
            if quarter_n < self.delay_qtrs:
                values.append(0)
            else:
                multiplier = -1 if self.is_cost else 1
                # TODO: multiply by -1 here if it is a COST we are considering
                values.append(multiplier * discounted_f(quarter_n))
        return values

    def quick_view(self):
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(range(self.tot_qtrs), self.sigmoid_qtr, label='sigmoid')
        ax.plot(range(self.tot_qtrs), self.linear_qtr, label='linear')
        ax.plot(range(self.tot_qtrs), self.step_qtr, label='step')
        ax.scatter(range(self.tot_qtrs), self.single_qtr, label='single')
        ax.legend(loc='upper left')
        ax.grid(True)
    
    @property
    def qtr(self):
        """calculates quarter for instance based on set function type"""
        return self._calculate_qtr(getattr(self, f'_{self.function.lower()}'))
    
    @property
    def sigmoid_qtr(self):
        return self._calculate_qtr(self._sigmoid)
    
    @property
    def linear_qtr(self):
        return self._calculate_qtr(self._linear)
    
    @property
    def step_qtr(self):
        return self._calculate_qtr(self._step)
    
    @property
    def single_qtr(self):
        return self._calculate_qtr(self._single)
    
    def to_json(self):
        return {
            "delay_qtrs": self.delay_qtrs,
            "discount_rate": self.discount_rate,
            "flow_id": str(self.id),
            "function": self.function,
            "is_cost": self.is_cost,
            "name": self.name,
            "max_amt": self.max_amt,
            "scale_up_qtrs": self.scale_up_qtrs,
            "tot_qtrs": self.tot_qtrs,
        }


In [3]:
class Styles():
    CONTAINER = "padding: 0; display: flex;"
    PARAM = "flex: 1; list-style-type: none; text-align: center;"
    VALUE = "display: block; text-align: center;"
    
class Project():
    """Add cash flows to a project"""
    def __init__(self, name, cash_flows=None):
        self.cash_flows = []
        self.name = name  # define before attempting to load from disk
        
        # If we pass in cash_flows as an argument, don't allow widgets to delete values (yet)
        self.read_only = cash_flows is not None
        if not self.read_only:
            self._load_from_disk()  # important to load before generating cash_flow_list
        else:
            self.cash_flows = cash_flows
            
        self.cash_flow_list = widgets.VBox(children=[self.widgetize_cf(cf) for cf in self.cash_flows])

        self.is_cost = widgets.Checkbox(value=False, description='Check if cash flow is a cost')
        self.cf_name = widgets.Text(placeholder='Name or description', description='Name')
        self.func_type = widgets.ToggleButtons(options=['linear', 'sigmoid', 'single', 'step'], description='Form')
        self.ben_delay = widgets.IntSlider(min=0, max=8, step=1, value=2, description='Ben. Delay', continuous_update=False)
        self.ben_ramp = widgets.FloatSlider(min=2, max=12, step=0.5, value=4, description='Ben. Ramp', continuous_update=False)
        self.ben_max = widgets.FloatSlider(min=0, max=30, step=0.5, value=4, description='Ben. Max', continuous_update=False)
        
        create_btn = widgets.Button(description="Add Cash Flow", disabled=self.read_only)
        create_btn.on_click(self.on_create_click)

        self.graph_out = widgets.Output()
        self.update_graphs()
        menu = widgets.VBox(
            children=[self.cf_name, self.is_cost, self.func_type, self.ben_delay, self.ben_ramp,
                      self.ben_max, create_btn, self.graph_out],
            layout=widgets.Layout(overflow="hidden")
        )
        
        self.app = widgets.AppLayout(left_sidebar=menu, right_sidebar=self.cash_flow_list)
        display(self.app)

    def _on_delete_click_factory(self, model):
        """A partially applied function that accepts a model,
        and returns a new function a button can click with context
        to a specific model"""

        def on_delete_click(button):
            index = None
            for i, cash_flow in enumerate(self.cash_flows):
                if cash_flow.id == model.id:
                    index = i
                    break

            if index is not None:
                self.cash_flows.pop(index)
                self.update_cash_flows()
                self.graph_out.clear_output(wait=True)
                self.update_graphs()

        return on_delete_click
    
    def _save_to_disk(self):
        with open(self.filename, 'w') as outfile:
            json.dump([cf.to_json() for cf in self.cash_flows], outfile)
    
    def _load_from_disk(self):
        try:
            with open(self.filename) as json_file:
                data = json.load(json_file)
                if not isinstance(data, list):
                    raise Exception(f'delete the file: {self.filename}, data is bad. It will regenerate.')
                for d in data:
                    cf = CashFlow(**d)
                    self.cash_flows.append(cf)
        except FileNotFoundError:
            print('could not find file, creating new')
    
    @property
    def filename(self):
        slug = self.name.lower().replace(' ', '-')
        return f'./data/{slug}-data.json'
        
    def update_cash_flows(self):
        self.cash_flow_list.children = [self.widgetize_cf(cf) for cf in self.cash_flows]
        self._save_to_disk()

    def on_create_click(self, event):
        f = CashFlow(
            delay_qtrs = self.ben_delay.value, 
            max_amt = self.ben_max.value,
            scale_up_qtrs = self.ben_ramp.value,
            function = self.func_type.value,
            name = self.cf_name.value,
            is_cost = self.is_cost.value
        )
        self.cash_flows.append(f)
        self.update_cash_flows()
        self.update_graphs()

    def widgetize_cf(self, model):
        html = f"""
        <h4 style="text-align: center">{model.name}</h1>
        <ul style="{Styles.CONTAINER}">
            <li style="{Styles.PARAM}">Function <strong style="{Styles.VALUE}">{model.function}</strong></li>
            <li style="{Styles.PARAM}">Is Cost <strong style="{Styles.VALUE}">{model.is_cost}</strong></li>
            <li style="{Styles.PARAM}">delay quarters <strong style="{Styles.VALUE}">{model.delay_qtrs}</strong></li>
            <li style="{Styles.PARAM}">delay quarters <strong style="{Styles.VALUE}">{model.discount_rate}</strong></li>
            <li style="{Styles.PARAM}">max amount <strong style="{Styles.VALUE}">{model.max_amt}</strong></li>
            <li style="{Styles.PARAM}">scale time <strong style="{Styles.VALUE}">{model.scale_up_qtrs}</strong></li>
        </ul>
        """
        html = widgets.HTML(
            value=html,
            layout=widgets.Layout(width='100%')
        )
        delete_button = widgets.Button(
            description='delete',
            button_style='danger',
            layout=widgets.Layout(margin='5px'),
            disabled=self.read_only,
        )
        delete_button.on_click(self._on_delete_click_factory(model))
        return widgets.VBox(
            children=[html, delete_button],
            layout=widgets.Layout(
                border='1px solid',
                margin='4px',
                justify_content='center',
                align_items='center',
                display='flex'
            )
        )
    
    def update_graphs(self):
        self.graph_out.clear_output(wait=True)
        if len(self.cash_flows) == 0:
            return

        def trim_axs(axs, N):
            axs = axs.flat
            for ax in axs[N:]:
                ax.remove()
            return axs[:N]

        def get_min_max(flows):
            values = reduce(list.__add__, [flow.qtr for flow in flows])
            minimum = min(0, min(values))
            maximum = max(values)
            return minimum + minimum * 0.2, maximum + maximum * 0.2  # + 20% for readability
        
        # capture stdout of matplotlib to output catching context graph_out as widget to include
        with self.graph_out:
            cols = 3
            rows = len(self.cash_flows) // cols + 1
            figsize = (10, 8)
            fig, axs = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)
            axs = trim_axs(axs, len(self.cash_flows))
            ymin, ymax = get_min_max(self.cash_flows)
            for ax, cf in zip(axs, self.cash_flows):
                ax.set_title(cf.name)
                ax.set_ylim([ymin,ymax])
                ax.grid(True)
                ax.xaxis.set_ticks(range(1, 13))
                ax.plot(range(12), cf.qtr)

            plt.show(fig)


### Project Data

In [4]:
access_token = 'u1zwu2cf2708fdfuy6y46ttwx3'
sheet_id = 3317327430215556
smartsheet
ss = smartsheet.Smartsheet(access_token)
sheet = ss.Sheets.get_sheet(sheet_id)

In [5]:
# Simple named tuple to organize the index of a cell, and if it is required or not
#
# Parsing immediately, and with intent of what must be there prevents
# a slew of boundary type errors or runtime ones.
Cell = namedtuple('Cell', ['index', 'name', 'required'])


class SmartsheetRow():
    """Reusable Smartsheet class for parsing a row. To use, define a child class which contains
    class attributes with a namedtuple defining each cell that should be parsed. This child class will
    inherit from this one, and automatically load and process all defined attributes.
    It is also possible to define methods to override the value when necessary. For example, divide an
    annual value by four"""
    def __init__(self, row):
        self.sheet_row = row
        self.cells = row['cells']
        self.row_number = row['rowNumber']
        # essentially grab the class attributes
        self.cell_defs = [getattr(self, attr) for attr in dir(self) if 'CELL_' in attr]
        self._load_cells()

    def _load_cells(self):
        for cell_def in self.cell_defs:
            modifier = getattr(self, f'_{cell_def.name}', None) or self._identity
            unmodified = self._get_cell(cell_def)
            value = modifier(unmodified) if unmodified is not None else None
            setattr(self, cell_def.name, value)
        
    @staticmethod
    def _identity(x):
        return x
        
    def _get_cell(self, cell_descriptor):
        """Will try to grab the cell regardless, if it is required, an error will be logged"""
        try:
            if cell_descriptor.required:
                cell_value = self.cells[cell_descriptor.index]['value']
                return cell_value
            else:
                cell = self.cells[cell_descriptor.index]
                return cell.get('value', None)
        except:
            raise Exception(f'Failed to process Row <{self.row_number}>. Missing {cell_descriptor.name}')

    def to_json(self):
        cell_defs = [getattr(self, attr) for attr in dir(self) if 'CELL_' in attr]
        return { cd.name: getattr(self, cd.name) for cd in cell_defs}


class PortfolioSheetRow(SmartsheetRow):
    """All cells that are desired MUST have a class attribute defined where:
       "CELL_" is part of the name. The class will use those attributes to parse the row.
       
       Any methods defined with the same "name" as the attribute preceeded by an underscore
       will apploy that function to the attribute as a clean way to modify the value if necessary.
       """
    CELL_DESC = Cell(0, 'name', True)
    CELL_FTE = Cell(1, 'fte', False)
    CELL_FTE_UNALLOCATED = Cell(2, 'fte_unallocated', False)
    CELL_FTE_OTHER = Cell(3, 'fte_other', False)
    CELL_INCLUDE_IN_MODEL = Cell(4, 'include_in_model', True)
    CELL_PROJECT_CODE = Cell(5, 'project_code', True)
    CELL_REV = Cell(6, 'annual_revenue', False)
    CELL_GP = Cell(7, 'gross_profit_perc', False)
    CELL_ATTRIB = Cell(8, 'attribution_perc', False)
    CELL_COST_OR_BENEFIT = Cell(9, 'is_cost', True)
    CELL_CF_PROFILE = Cell(10, 'function', True)
    CELL_DISCOUNT_RATE = Cell(11, 'discount_rate', True)
    CELL_ANNUALIZED_START_VALUE = Cell(12, 'start_value', False)
    CELL_DELAY_QUARTERS = Cell(13, 'delay_qtrs', True)
    CELL_ANNUALIZED_MAX_VALUE = Cell(14, 'max_amt', True)
    CELL_RAMP_TIME = Cell(15, 'scale_up_qtrs', True)
    CELL_COMMENTS = Cell(16, 'comments', False)
    
    def __init__(self, row):
        # Extra logic is used to decide if row should be processed at all
        # This allows us to "fail fast" when a row we want to parse doesn't have the data we want
        cells = row['cells']
        include = cells[PortfolioSheetRow.CELL_INCLUDE_IN_MODEL.index].get('value', None)
        self.project_code = cells[PortfolioSheetRow.CELL_PROJECT_CODE.index].get('value', None)
        self.include_in_model = True if include == "Yes" else False
        
        if self.include_in_model and self.project_code:
            super().__init__(row)
        
    @staticmethod
    def _function(val):
        text = val.strip().lower()
        if text == 'continuous' or text == 'step':
            return 'step'
        elif text == 'single pmt.':
            return 'single'
        elif text == 'logistic':
            return 'sigmoid'
        elif text == 'linear':
            return 'linear'
        else:
            raise Exception(f'Unknown profile type: {val}')
    
    @staticmethod
    def _include_in_model(val):
        return val.lower() == 'yes'

    @staticmethod
    def _is_cost(val):
        return val.lower() == 'cost'
    
    

# Script for quick verification testing
# row_num = 21
# row = sheet.rows[row_num - 1]
# r = PortfolioSheetRow(row.to_dict())
# blacklisted = ['__', 'cell_defs', 'CELL', 'cells', 'sheet_row', '_get_cell', '_identity', '_profile_type', 'row_number', '_include']
# for a in dir(r):
#     if any([s in a for s in blacklisted]):
#         continue
#     print(a + '\t\t\t' + str(getattr(r, a)))

### Quarter Summary

In [29]:
# Allow for easy targeting of rows, here [20 - 70]
SHEET_ROW_START = 20
SHEET_ROW_END = 70

def sheet_row_to_cf(row):
    return CashFlow(
        delay_qtrs = row.delay_qtrs,
        discounted = True,
        discount_rate = row.discount_rate,
        max_amt = row.max_amt,
        scale_up_qtrs = row.scale_up_qtrs,
        function = row.function,
        name = row.name,
        is_cost = row.is_cost
    )

# iterate through smartsheet and build project map
project_map = defaultdict(list)
for index, sheet_row in enumerate(sheet.rows):
    if index < SHEET_ROW_START or index > SHEET_ROW_END:
        continue

    row = PortfolioSheetRow(sheet_row.to_dict())
    if row.project_code and row.include_in_model:
        project_map[row.project_code].append(sheet_row_to_cf(row))

# Build DataFrame from project_map
rows = []
for project_id, cash_flows in project_map.items():
    for cash_flow in cash_flows:
        # Discounted
        assert cash_flow.discounted
        rows.append([project_id, True, cash_flow.name] + cash_flow.qtr)
        # Undiscounted
        cash_flow.discounted = False
        rows.append([project_id, False, cash_flow.name] + cash_flow.qtr)
        
df = pd.DataFrame(data=rows, columns=['project', 'discounted', 'name'] + quarter_labels)
df.groupby(['project', 'discounted']).sum(numeric_only=True).sum(numeric_only=True, axis=1)

project  discounted
BP       False         -4.678182
         True          -4.746166
FS       False         13.360683
         True           7.260043
PV       False         12.386509
         True           3.458543
dtype: float64

In [22]:
def display_dataframe(df, title):
    display(HTML(f'<div><h3>{title}</h3></div>'))
    df.loc["Total"] = df.sum(numeric_only=True)
    cleaned = df.drop(['project', 'discounted'], axis=1)
    display(cleaned)

    
for project_id in df['project'].unique():
    display(HTML(f'<div><h1>{project_id}</h1><hr /></div>'))
    ndf = df[df['project'] == project_id]
    display_dataframe(ndf[ndf['discounted']].copy(), "Discounted Cash Flows")
    display_dataframe(ndf[ndf['discounted'] == False].copy(), "Undiscounted Cash Flows")

,name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
0,Frontsight Program Costs,-0.3125,-0.304878,-0.297442,-0.290187,-0.283110,-0.276204,-0.269468,-0.262895,-0.256483,-0.250228,-0.244125,-0.238170
2,Pipeline FS Product Credits,-0.1800,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
4,Pipeline FS - Stockton,0.0000,0.000000,0.000000,0.000000,0.122942,0.111766,0.101605,0.092368,0.083971,0.076338,0.069398,0.063089
6,Pipeline FS - Magic Valley,0.0000,0.000000,0.136106,0.118353,0.102916,0.089492,0.077819,0.067669,0.058842,0.051167,0.044493,0.038690
8,Projected Whitespace Wins,0.0000,0.000000,0.000000,0.129576,0.244355,0.438631,0.724998,1.066022,1.372924,1.572964,1.659398,1.666336
10,Projected Margin Accretion,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011193,0.021108,0.037891,0.062628,0.092087,0.118598
Total,NaN,-0.4925,-0.304878,-0.161336,-0.042259,0.187103,0.363684,0.646147,0.984272,1.297145,1.512869,1.621252,1.648543


,name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
1,Frontsight Program Costs,-0.3125,-0.3125,-0.3125,-0.3125,-0.312500,-0.312500,-0.312500,-0.312500,-0.312500,-0.312500,-0.312500,-0.312500
3,Pipeline FS Product Credits,-0.1800,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Pipeline FS - Stockton,0.0000,0.0000,0.0000,0.0000,0.180000,0.180000,0.180000,0.180000,0.180000,0.180000,0.180000,0.180000
7,Pipeline FS - Magic Valley,0.0000,0.0000,0.1800,0.1800,0.180000,0.180000,0.180000,0.180000,0.180000,0.180000,0.180000,0.180000
9,Projected Whitespace Wins,0.0000,0.0000,0.0000,0.1500,0.297015,0.559816,0.971566,1.500000,2.028434,2.440184,2.702985,2.850000
11,Projected Margin Accretion,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.015000,0.029702,0.055982,0.097157,0.150000,0.202843
Total,NaN,-0.4925,-0.3125,-0.1325,0.0175,0.344515,0.607316,1.034066,1.577202,2.131915,2.584840,2.900485,3.100343


,name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
12,External Investment,-0.000000,-0.048780,-0.095181,-0.139290,-0.181190,-0.220964,-0.258689,-0.294443,-0.328299,-0.360328,-0.390599,-0.419180
14,Internal Headcount,-0.593182,-0.578714,-0.564599,-0.550828,-0.537393,-0.524286,-0.511499,-0.499023,-0.486852,-0.474978,-0.463393,-0.452090
16,Revenue Defense - Showme,0.200000,0.192771,0.185803,0.179088,0.172615,0.166376,0.160362,0.154566,0.148979,0.143594,0.138404,0.133402
18,United Ethanol Whitespace Win,0.120000,0.115663,0.111482,0.107453,0.103569,0.099825,0.096217,0.092739,0.089387,0.086157,0.083042,0.080041
20,Guardian - Hankinson Defense,0.000000,0.000000,0.000000,0.000000,0.000000,0.147702,0.139013,0.130836,0.123140,0.115896,0.109079,0.102662
22,Valero - Net new,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033932
24,Madeira - Fleet Trials,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057948,0.054539,0.051331
Total,NaN,-0.273182,-0.319061,-0.362495,-0.403578,-0.442400,-0.331347,-0.374595,-0.415325,-0.453644,-0.431710,-0.468927,-0.469902


,name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
13,External Investment,-0.000000,-0.050000,-0.100000,-0.150000,-0.200000,-0.250000,-0.300000,-0.350000,-0.400000,-0.450000,-0.500000,-0.550000
15,Internal Headcount,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182,-0.593182
17,Revenue Defense - Showme,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000
19,United Ethanol Whitespace Win,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000,0.120000
21,Guardian - Hankinson Defense,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000
23,Valero - Net new,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000
25,Madeira - Fleet Trials,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.100000
Total,NaN,-0.273182,-0.323182,-0.373182,-0.423182,-0.473182,-0.323182,-0.373182,-0.423182,-0.473182,-0.423182,-0.473182,-0.323182


,name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
26,Internal Headcount,-0.444886,-0.434035,-0.423449,-0.413121,-0.403045,-0.393215,-0.383624,-0.374267,-0.365139,-0.356233,-0.347545,-0.339068
28,Benefits - Contracted GPRE,0.375000,0.352941,0.332180,0.312640,0.294249,0.276941,0.260650,0.245318,0.230887,0.217306,0.204523,0.192492
30,Benefits - Potential GPRE,0.000000,0.000000,0.000000,0.158025,0.222410,0.307521,0.414211,0.537858,0.665971,0.779489,0.859559,0.896002
Total,NaN,-0.069886,-0.081094,-0.091269,0.057543,0.113614,0.191246,0.291237,0.408909,0.531719,0.640561,0.716537,0.749427


,name,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
27,Internal Headcount,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886,-0.444886
29,Benefits - Contracted GPRE,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
31,Benefits - Potential GPRE,0.000000,0.000000,0.000000,0.225000,0.356258,0.554162,0.839725,1.226692,1.708737,2.250000,2.791263,3.273308
Total,NaN,-0.069886,-0.069886,-0.069886,0.155114,0.286372,0.484276,0.769838,1.156806,1.638850,2.180114,2.721377,3.203421


In [30]:
# Build UI from project_map
projects = []
for project_id, cash_flows in project_map.items():
    display(HTML(f'<div><h1>{project_id}</h1><br /><hr /></div>'))
    projects.append(Project(project_id, cash_flows))

AppLayout(children=(VBox(children=(Text(value='', description='Name', placeholder='Name or description'), Chec…

AppLayout(children=(VBox(children=(Text(value='', description='Name', placeholder='Name or description'), Chec…

AppLayout(children=(VBox(children=(Text(value='', description='Name', placeholder='Name or description'), Chec…

In [10]:
def combine_flows(flows):
    qtr_values = map(lambda cf: cf.qtr, flows)
    aggregated_values = [sum(values) for values in zip(*qtr_values)]
    return aggregated_values

# combine_flows(projects[0].cash_flows)

In [11]:
graph_out = widgets.Output()

def combine_flows(flows):
    qtr_values = map(lambda cf: cf.qtr, flows)
    aggregated_values = [sum(values) for values in zip(*qtr_values)]
    return aggregated_values

def get_x_ticks(index, bar_width, step=1):
    return [x - (bar_width * len(projects)) + (bar_width * index) for x in range(12)]

def graph_projects_cash_flow(_):
    graph_out.clear_output(wait=True)
    
    with graph_out:
        projects_values = map(lambda p: combine_flows(p.cash_flows), projects)
        projects_net = [sum(values) for values in zip(*projects_values)]
        
        figsize = (25, 8)
        plt.figure(figsize=figsize)
        plt.xticks(range(0, 12))
        bar_room = .75 # 75% of space will have a bar plotted on it
        bar_width = bar_room * 1 / (len(projects) + 1)  # + 1 in denominator to account for 'net'
        legend_parents = []
        for i, project in enumerate(projects):
            xseries = get_x_ticks(i, bar_width)
            plot = plt.bar(xseries, combine_flows(project.cash_flows), width=bar_width)
            legend_parents.append(plot)

        legend_parents = [p[0] for p in legend_parents]
        net_xseries = get_x_ticks(len(projects), bar_width)
        net_plot = plt.bar(net_xseries, projects_net, color='black', width=bar_width)

        legend_parents.append(net_plot[0])
        plt.legend((*legend_parents,), (*map(lambda p: p.name, projects), 'net'))
        plt.show()

with graph_out:
    graph_projects_cash_flow(None)

button = widgets.Button(description='Refresh Chart')
button.on_click(graph_projects_cash_flow)

chart = widgets.VBox(children=[button, graph_out])
chart

## Step 1

In [12]:
### STEP 1 START ###
# For logistic growth we can define a sigmoid function
# Sigmoid function, f(-6) = 0; f(0) = 0.5;  f(6) = 1.0;
# Sigmoid / logistic growth would be a common cash flow profile
# Future options to add would be 'step function,' 'linear growth,' 'exponential growth,' etc.
def sigmoid(x):
    shift = 3 # 5% of max value is "zero"
    return 1 / (1 + math.exp(-(x - shift)))

# Define a quarterly implementation of logistic growth / sigmoid based on delay in years, max $ amount, and the time to ramp up
# All values in quarters except the 'max_amt' which is in dollars($M USD)
# max_amt is the 'carrying capacity' of the sigmoid function, usually known as 'k' in academic formulas
def sigmoid_qtr(delay_qtrs, max_amt, scale_up_qtrs, tot_qtrs=12):
    mylist = []
    if scale_up_qtrs < 2: 
        print('the total number of quarters must be at least one')
    # -3 corresponds to 5% of max value, and 3 corresponds to 95% of max value, going to use those values to define function.
    sigmoid_start = -3
    sigmoid_last = 3
    quarterly_x = (sigmoid_last - sigmoid_start) / scale_up_qtrs
    # print(f'quarterly_x = {quarterly_x}')
    for i in range(0, tot_qtrs):
        if i < delay_qtrs:
            mylist.append(0)
        else: 
            x_val = sigmoid_start + quarterly_x*(i - delay_qtrs)
            mylist.append(max_amt * sigmoid(x_val))
    return mylist

# Let's try a test case with some benefits with a max of $5M that start in 3 quarters, fully realized in 6 quarters
delay_qtrs = 3
max_amt = 5
scale_up_qtrs = 7
test = sigmoid_qtr(delay_qtrs, max_amt, scale_up_qtrs)

# Red line is the number of delay quarters, when the function kicks in
# Blue line is the end of scale up when you are getting close to max benefits
def plotter(ben_vals, delay_qtrs, scale_up_qtrs):
    labels = np.arange(1, len(test)+1)
    plt.bar(labels, ben_vals)
    plt.axvline(x=delay_qtrs, color = 'r')
    plt.title('Scale up of Cash Flows')
    plt.axvline(x=(delay_qtrs + scale_up_qtrs), color = 'b')
    plt.show;
# plotter(test, delay_qtrs, scale_up_qtrs)

# This should print both costs and benefits
def dual_plotter(costs, bennies):

    # set width of bar
    barWidth = 0.25

    # calculate the net benefit for the quarter
    net = [x1 - x2 for (x1, x2) in zip(bennies, costs)]
    
    # set height of bar
    bars1 = costs
    bars2 = bennies
    bars3 = net

    # Set position of bar on X axis
    r1 = np.arange(len(bars1))
    r2 = [x + barWidth for x in r1]
    r3 = [x + barWidth for x in r2]

    # Make the plot
    plt.bar(r1, bars1, color='r', width=barWidth, edgecolor='white', label='cost')
    plt.bar(r2, bars2, color='b', width=barWidth, edgecolor='white', label='bennies')
    plt.bar(r3, bars3, color='#2d7f5e', width=barWidth, edgecolor='white', label='net')

    # Add xticks on the middle of the group bars
    plt.xlabel('group', fontweight='bold')
    plt.xticks([r + barWidth for r in range(len(bars1))], quarter_labels)

    # Create legend & Show graphic
    plt.legend()
    plt.show();


In [13]:
def f(x, y, z, a, b, c):
    benefits = sigmoid_qtr(x, y, z)
    costs = sigmoid_qtr(a, b, c)
    
    # To plot a single cash flow:
    # plotter(benefits, x, y, costs)
    
    # To plot both costs and benefits together:
    dual_plotter(benefits, costs)


# Need to get this working for negative numbers.  Cash flow profiles for costs / investments will be negative numbers.
# But maybe we can just calculate as positive, and then take the diff for net cash flow and put into a dataframe
# Just need to review math of the above functions to make sure it works for negative numbers.
# Turning off continuous update here :) 
benefits_delay = widgets.IntSlider(min=0, max=8, step=1, value=2, description='Ben. Delay', continuous_update=False)
benefits_ramp = widgets.FloatSlider(min=2, max=12, step=0.5, value=4, description='Ben. Ramp', continuous_update=False)
benefits_max = widgets.FloatSlider(min=0, max=30, step=0.5, value=4, description='Ben. Max', continuous_update=False)

# We should probably include a component eventually where costs are a function of benefits, but for now, let's model them independently.
cost_delay = widgets.FloatSlider(min=0, max=12, step=0.5, value=4, description='Cost Delay', continuous_update=False)
cost_ramp = widgets.FloatSlider(min=0, max=12, step=0.5, value=4, description='Cost Ramp', continuous_update=False)
cost_max = widgets.FloatSlider(min=0, max=30, step=0.5, value=4, description='Cost Max', continuous_update=False)

interact(f, x=benefits_delay, y=benefits_ramp, z=benefits_max, a=cost_delay, b=cost_ramp, c=cost_max);


##### There is something weird in how the sigmoid_qtr function is returning values.  
##### Example, if you set the 'cost max' or 'benefit max' really high, it seems to change the ramp-up time, 
##### and the max value in the slider doesn't match the max value in the plot.

interactive(children=(IntSlider(value=2, continuous_update=False, description='Ben. Delay', max=8), FloatSlide…

# Step 2

In [14]:
## Discounted Cash Flows can use this formula, once we find the : 
example_net_cashflow = [-100, 19, 49, 58, 200]

# Using a shortcut here dividing by four to calculate quarterly interest from annual discount rate of 25%
# There is a better way to do the conversion, but example below:
npv_net =  np.npv(0.35/4,[-100, 19, 49, 58, 200, 100]) 
print("Net Present Value(npv) : ", npv_net) 

# we could also calculate an IRR using the IRR module from python, but NPV is fine for now.

#### STEP 2 END
#### STEP 3 START

Net Present Value(npv) :  212.7356691470436


In [15]:
## Start building out a dataframe for project financial parameters and 12 months of cash flows
## Cash flows can be populated with the growth curves above

row_titles = ['visibility', 'growth', 'gamechangers']
column_titles = ['internal_fte', 'external_spend', 'max_benefit', 'profile', 'discount_rate']
d = pd.DataFrame(0, index=row_titles, columns=column_titles)

In [16]:
quarter_labels

['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12']

In [17]:
years_in_model = 3
quarters = (list(range(years_in_model * 4)))
quarters = [q + 1 for q in quarters]

In [18]:
a = np.zeros((len(row_titles),len(quarters)))
cost_benefit = pd.DataFrame(a, index = row_titles, columns=quarters)

In [19]:
# so once we estimate the benefits and the costs, we can populate the quarterly cash flows into a dataframe
# The three major categories of projects are visibility (R2K, etc.), growth (batch perf., etc.), gamechangers (proteinview, etc.)
comb = pd.concat([d, cost_benefit], axis=1, sort=False)
comb

# Once we phase in cash flows, we can discount them back with an Net Present Value (NPV) using the discount rate.

,internal_fte,external_spend,max_benefit,profile,discount_rate,1,2,3,4,5,6,7,8,9,10,11,12
visibility,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
growth,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gamechangers,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
### HMM... how to iterate over the rows to replace the cash flows in the data frame for a given investment area.
# adder = 1
# for key, value in cost_benefit.iteritems(): 
#     value = value + adder
#     adder += 1
#     print(value) 
#     print() 